# How biased are the simulations?

In [ ]:
chunk_start

In [ ]:
# model_dict = {
#     "LOCATE":"/data/proteus1/scratch/gle/getmval/locate",
#     "GETM":"/data/proteus1/scratch/rwi/adhoc/getm/3dmn"
# }

model_dict = model_dict_str
num_models = len(model_dict)

In [ ]:
annual_paths = []
for key in model_dict:
    if not os.path.exists(model_dict[key] + "/results/annual_mean/"):
        raise ValueError("No annual mean folder found for " + key)
    paths = glob.glob(model_dict[key] + "/results/annual_mean/*")
    paths = tidy_summary_paths(paths)
    annual_paths.append(
        pd.DataFrame({"path": paths})
        .assign(model = key)
    )


annual_paths = pd.concat(annual_paths)

annual_paths["base_name"] = annual_paths["path"].apply(lambda x: os.path.basename(x))
# only interested in netcdf file in path
annual_paths = annual_paths[annual_paths["base_name"].str.contains(".nc")]
# fix base_name using fix_base_name
annual_paths["base_name"] = annual_paths["base_name"].apply(fix_basename)




In [ ]:
annual_paths = (
    annual_paths
    .groupby("base_name")
    .count()
      .query("model > 1")
      .reset_index()
      .drop(columns = ["path", "model"])
      .merge(annual_paths)
    )

In [ ]:
base_names = annual_paths.base_name.unique() 

In [ ]:
output = dict()
md_output = dict()
text_output = dict()
# list to track data frames with correlation coefficients
df_cor = []
df_abs = []
for bb in base_names:
    variable = bb.split("_")[1].replace(".nc", "")
    bb_paths = annual_paths.query("base_name == @bb").reset_index(drop = True)
    n_cols = len(bb_paths)
    # generate the mask first

    ds_mask = nc.open_data(bb_paths.path[0])
    ds_mask.run()
    for ff in bb_paths.path[1:]:
        ds_ff = nc.open_data(ff)
        ds_ff.regrid(ds_mask)
        ds_mask * ds_ff
        ds_mask.run()
        ds_mask.abs()
        ds_mask > 0
        ds_mask.run()
    df_mask = (
        ds_mask.to_dataframe()
        .dropna()
        .reset_index()
    )
    lon_name = [x for x in df_mask.columns if "lon" in x][0]
    lat_name = [x for x in df_mask.columns if "lat" in x][0]
    # rename 
    df_mask = df_mask.rename(columns = {lon_name: "lon", lat_name: "lat"})

    lon_min = df_mask.lon.min()
    lon_max = df_mask.lon.max()
    lat_min = df_mask.lat.min()
    lat_max = df_mask.lat.max()
    lons = [lon_min, lon_max]
    lats = [lat_min, lat_max]
    # coerse to float
    lons = [float(x) for x in lons]
    lats = [float(x) for x in lats]
    ds_mask.subset(lon = lons, lat = lats)
    ds_mask.run()
    import matplotlib.pyplot as plt

    plt.subplots_adjust(wspace=20, hspace=20)

    fig = plt.figure(figsize=(10, 10))

    # Create 4x4 Grid
    
    key = md(f"## How biased is surface {fix_variable_name(variable)}?")

    gs = fig.add_gridspec(nrows=1, ncols=num_models + 1, wspace = 0.6, hspace = 0)
    # get the minimum and maximum values for the colorbar

    z_max = -1
    z_min = 1





    for i in range(0, len(bb_paths)):
        ds = nc.open_data(bb_paths.path[i])
        ds.regrid(ds_mask, "nn")
        ds * ds_mask
        ds.assign(bias = lambda x: x.model - x.observation)
        ds.run()
        i_max = ds.to_dataframe().dropna().reset_index().bias.quantile(0.98)
        i_min = ds.to_dataframe().dropna().reset_index().bias.quantile(0.02)
        if i_max > z_max:
            z_max = i_max
        if i_min < z_min:
            z_min = i_min


    for i in range(0, len(bb_paths)):
        ds = nc.open_data(bb_paths.path[i])
        unit  = ds.contents.unit[0]
        ds.regrid(ds_mask, "nn")
        ds * ds_mask
        ds.run()
        ds.assign(bias = lambda x: x.model - x.observation, drop = True)
        ds.set_longnames({"bias": "Bias"})
        ds.set_units({"bias": unit})
        #get the model run name
        model_name = bb_paths.model[i]
        ds.to_latlon(lon = lons, lat = lats, res = [0.111, 0.067])
        ds.pub_plot(  fig = fig, gs = gs[0,i], title = model_name, limits = [z_min, z_max])
        ds_abs = ds.copy()
        ds.spatial_mean()
        cor_value = ds.to_dataframe().dropna().reset_index().bias[0]
        # stick this in a dataframme
        df_cor.append(
            pd.DataFrame({"model": [model_name], "variable": variable, "bias": cor_value})
        )
        ds_abs.abs()
        ds_abs.run()
        ds_abs.spatial_mean()
        # calculate the normalized bias
        ds = nc.open_data(bb_paths.path[i])
        ds.to_latlon(lon = lons, lat = lats, res = [0.111, 0.067]) 
        ds.assign(bias = lambda x: x.model - x.observation)
        ds.drop(variable = "model")
        ds.cell_area(join = True)
        ds.assign(bias = lambda x: x.bias * x.cell_area)
        ds.assign(observation = lambda x: x.observation * x.cell_area)
        ds.spatial_sum()
        ds.assign(bias = lambda x: x.bias / x.observation, drop = True)
        norm_bias = ds.to_dataframe().dropna().reset_index().bias[0]
        df_abs.append(
            pd.DataFrame({"model": [model_name], "variable": variable, "bias": ds_abs.to_dataframe().dropna().reset_index().bias[0]})
            .assign(unit = unit)
            .assign(norm_bias = norm_bias)
        )


    output[key] = fig 

    ds_diff = nc.open_data(bb_paths.path[0])
    ds_diff.subset(variable = "model")
    ds1 = nc.open_data(bb_paths.path[1])
    ds1.subset(variable = "model")
    ds1.regrid(ds_diff)
    ds_diff - ds1
    ds_diff.regrid(ds_mask, "nn")
    ds_mask.subset(variable = "model")
    ds_diff * ds_mask
    ds_diff.run()
    model1 = bb_paths.model[0]
    model2 = bb_paths.model[1]
    title = f"{model1} - {model2}"
    ds_diff.set_longnames({ds_diff.variables[0]: title})
    ds_diff.to_latlon(lon = lons, lat = lats, res = [0.111, 0.067]) 
    ds_diff.pub_plot(  fig = fig, gs = gs[0,2], title = title, limits = ["2%", "98%"])


    md_output[key] = md(f"**Figure {i_figure}**: Model bias for surface {fix_variable_name(variable)}. The first two columns show **model - observation** for the two simulations. The third column shows the difference between the two models.")
    i_figure += 1
    try:
        name1 = bb_paths.base_name[0].split(variable)[1].split("_")[1].replace(".nc", "")
        name2 = bb_paths.base_name[1].split(variable)[1].split("_")[1].replace(".nc", "")
    except:
        name1 = "foo"
        name2 = "bar"
    comp_text = None
    if name1 == name2:
        if len(name1) > 0:
            source = name1.upper()
            comp_text = f"Bias was calculated by comparing annual averages in the simulation with the observational data from **{source}**."
    text_output[key] = comp_text
    

In [ ]:
for key in output:
    key
    if text_output[key] is not None:
        display(md(text_output[key]))
    display(output[key])
    display(md_output[key])

In [ ]:
df_cor = pd.concat(df_cor)
df_abs = pd.concat(df_abs)

## Overall summary of model biases

In [ ]:
# spread model and cor in columns
# df_cor.pivot(index = "variable", columns = "model", values = "bias")

df_out = (
    df_cor.pivot(index = "variable", columns = "model", values = "bias")
    .reset_index()
    .merge(
    df_abs
    .loc[:,["variable", "unit"]]
    .drop_duplicates()
)
)
df_display(df_out)

In [ ]:
md(f"**Table {i_table}**: Mean bias for each model and variable") 
i_table += 1

To make biases more comparable, the table below summarizes the normalized biases of each simulation, using the equation:

$$
\text{Normalized bias} = \frac{\sum_{i=1}^{n} \left| M_i - O_i \right|}{\sum_{i=1}^{n} O_i}

$$

where $M_i$ is the model output, $O_i$ is the observed value, and $n$ is the number of data points. The normalized bias is a measure of how much the model output deviates from the observed value, relative to the observed value. A normalized bias of 0 means that the model output is exactly the same as the observed value, while a normalized bias of 1 means that the model output is twice as large as the observed value.



In [ ]:
df_out = df_abs.pivot(index = "variable", columns = "model", values = "norm_bias").reset_index()
df_display(df_out)
md(f"**Table {i_table}**: Normalized bias for each model and variable")
i_table += 1

The table below shows mean absolute error for each model and variable.

In [ ]:
df_out = df_abs.pivot(index = "variable", columns = "model", values = "bias").reset_index()
# display the table without index
df_display(df_out)

In [ ]:
md(f"**Table {i_table}**: Mean absolute error for each model and variable")
i_table += 1